In [116]:
import pandas

#load csv
open_line_list = pandas.read_csv('COVID19_open_line_list.csv')

In [117]:
#remove extra empty column
open_line_list = open_line_list.iloc[:,0:33]

#select only usefull columns
open_line_list = open_line_list[['age','sex','city','province','country','wuhan(0)_not_wuhan(1)','date_confirmation','symptoms','lives_in_Wuhan','travel_history_location','chronic_disease_binary','chronic_disease','outcome','date_death_or_discharge']]

#fix na values with 0
open_line_list['chronic_disease_binary']=open_line_list['chronic_disease_binary'].fillna(0)

In [118]:
#create new column with boolean values
open_line_list['travel_in_Wuhan']=open_line_list['travel_history_location'].str.contains("Wuhan",na=False, regex=True)

#change logic to column wuhan(0)_not_wuhan(1) and name to inWuhan
open_line_list['wuhan(0)_not_wuhan(1)']=1-open_line_list['wuhan(0)_not_wuhan(1)']
open_line_list.rename(columns={"wuhan(0)_not_wuhan(1)":"inWuhan"},inplace=True)
open_line_list['inWuhan']=open_line_list['inWuhan'].fillna(0)

#fillna lives_in_wuhan
open_line_list['lives_in_Wuhan']=open_line_list['lives_in_Wuhan'].fillna('no')
open_line_list['lives_in_Wuhan_binary']=open_line_list['lives_in_Wuhan']!='no'

In [119]:
#drop extra rows
index = open_line_list[open_line_list['country'].isnull()].index
open_line_list.drop(index,inplace=True)
open_line_list.drop(columns=["lives_in_Wuhan","travel_history_location","chronic_disease","outcome","date_death_or_discharge"],inplace=True)
indexes = open_line_list[open_line_list['age'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexes.tolist(),'age'] = ''
indexes = open_line_list[open_line_list['date_confirmation'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexes.tolist(),'date_confirmation'] = ''

In [120]:
open_line_list.to_csv('COVID19_open_line_list_clean.csv',index=False)